# Multilayer Perceptron for Taxi Prediction 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import copy
import random
import time

# RNG SEED

In [2]:
SEED = 12345

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
class CustomDataset(Dataset):
    def __init__(self, train):
        self.train_data = torch.tensor(pd.read_csv(train).astype('float32').values)
        self.targets = torch.tensor(pd.read_csv("targets_engineered.csv").astype('float32').values)

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return self.train_data[idx], self.targets[idx]

The first thing we'll do is load our dataset "train_engineered.csv" 

In [4]:
train_data = CustomDataset("train_engineered.csv")

In [6]:
train_data[0]

(tensor([  0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000, 691.6916,
           0.0000,  60.0000]),
 tensor([330.]))

In [7]:
print(f'Number of training examples: {len(train_data)}')

Number of training examples: 1710660


In [8]:
VALID_RATIO = 0.95

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

In [9]:
train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

TEST_RATIO = 0.95
n_valid_examples = int(len(valid_data) * VALID_RATIO)
n_test_examples = len(valid_data) - n_valid_examples
valid_data, test_data = data.random_split(valid_data,
                                           [n_valid_examples, n_test_examples])

We can print out the number of examples again to check our splits are correct.

In [10]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1625127
Number of validation examples: 81256
Number of testing examples: 4277


# BATCH SIZE

In [11]:
BATCH_SIZE = 6400

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

In [12]:
x, y = next(iter(train_iterator))
print(x)
print(y)

tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 7.6439e+02, 5.0000e+00,
         8.8100e+02],
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 6.5102e+02, 6.0000e+00,
         2.7600e+02],
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 8.2437e+02, 4.0000e+00,
         1.0210e+03],
        ...,
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 6.6139e+02, 6.0000e+00,
         2.5100e+02],
        [1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 6.2671e+02, 3.0000e+00,
         3.1000e+01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 6.6855e+02, 4.0000e+00,
         5.5200e+02]])
tensor([[ 240.],
        [ 465.],
        [1485.],
        ...,
        [ 285.],
        [1110.],
        [ 300.]])


In [13]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 1000)
        self.hidden_fc = nn.Linear(1000, 500)
        self.hidden_fc2 = nn.Linear(500, 250)
        self.hidden_fc3 = nn.Linear(250, 100)
        self.hidden_fc4 = nn.Linear(100, 50)
        self.output_fc = nn.Linear(50, output_dim)

    def forward(self, x):

        # x = [batch size, height, width]

        batch_size = x.shape[0]

        x = x.view(batch_size, -1)

        # x = [batch size, height * width]

        h_1 = F.relu(self.input_fc(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))
        h_2 = F.relu(self.hidden_fc2(h_2))
        h_2 = F.relu(self.hidden_fc3(h_2))
        h_2 = F.relu(self.hidden_fc4(h_2))

        # h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)

        # y_pred = [batch size, output dim]

        return y_pred, h_2

In [14]:
INPUT_DIM = 9
OUTPUT_DIM = 1

model = MLP(INPUT_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 665,951 trainable parameters


# LOSS FUNCTION

In [17]:
criterion = nn.MSELoss()

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
model = model.to(device)
criterion = criterion.to(device)

# OPTIMIZER

In [20]:
optimizer = optim.Adagrad(model.parameters())

In [21]:
def calculate_accuracy(y_pred, y):
    '''top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]'''
    return 0 #acc

In [22]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += 0 #acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += 0 #acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# NUMBER OF EPOCHS

In [25]:
EPOCHS = 100

best_valid_loss = float('inf')

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/254 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 464879.449 | Train Acc: 0.00%
	 Val. Loss: 391814.469 |  Val. Acc: 0.00%


Training:   0%|          | 0/254 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 99 | Epoch Time: 0m 21s
	Train Loss: 434233.426 | Train Acc: 0.00%
	 Val. Loss: 382817.002 |  Val. Acc: 0.00%


Training:   0%|          | 0/254 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 100 | Epoch Time: 0m 23s
	Train Loss: 434217.309 | Train Acc: 0.00%
	 Val. Loss: 382611.219 |  Val. Acc: 0.00%


Afterwards, we load our the parameters of the model that achieved the best validation loss and then use this to evaluate our model on the test set.

In [26]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 298224.656 | Test Acc: 0.00%


In [28]:
test_data = CustomDataset("test_engineered.csv")
test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

df_result = pd.read_csv("sampleSubmission.csv", index_col = False)

model.eval()

with torch.no_grad():

    for (x, y) in tqdm(test_iterator, desc="Evaluating", leave=False):

        x = x.to(device)
        y = y.to(device)

        y_pred, _ = model(x)
        
        df_result['TRAVEL_TIME'] = pd.DataFrame(y_pred.cpu().numpy())

df_result.to_csv("submission.csv", index=False)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
df_result

,TRIP_ID,TRAVEL_TIME
0,T1,753.456665
1,T2,679.738892
2,T3,598.511108
3,T4,602.600769
4,T5,597.122803
...,...,...
315,T323,746.238037
316,T324,660.576904
317,T325,1007.054321
318,T326,749.842285
